# 1. Setup Environment
First, let's install the necessary libraries:

In [1]:
# Install required libraries
!pip install tensorflow tensorflow-hub tensorflow-datasets
!pip install tf-slim

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# 2. Load the COCO MiniTrain Dataset
We'll now load the COCO MiniTrain dataset from TensorFlow Datasets.

In [ ]:
# Load COCO MiniTrain dataset (smaller subset)
train_dataset, info = tfds.load('coco/2017', split='train[:1%]', with_info=True)  # Use 1% of the dataset
val_dataset = tfds.load('coco/2017', split='validation[:1%]')

# Extract class names
class_names = info.features['objects']['label'].names


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

# 3. Preprocess Dataset
We will resize, normalize, and prepare bounding boxes and labels.

In [ ]:
# Preprocessing function for COCO MiniTrain
def preprocess(example):
    image = tf.image.resize(example['image'], (300, 300)) / 255.0  # Resize and normalize
    boxes = example['objects']['bbox']  # Bounding boxes
    labels = example['objects']['label']  # Class labels
    return image, boxes, labels

# Apply preprocessing and batch the data
train_dataset = train_dataset.map(preprocess).batch(4).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(4).prefetch(tf.data.AUTOTUNE)


# 4. Visualize Dataset Samples
Let's visualize a few samples from the dataset.

In [ ]:
# Function to display samples
def show_samples(dataset):
    plt.figure(figsize=(12, 4))
    for i, (image, boxes, labels) in enumerate(dataset.take(3)):
        plt.subplot(1, 3, i+1)
        plt.imshow(image[0].numpy())  # Display first image in the batch
        for box in boxes[0]:  # Draw boxes for first image
            ymin, xmin, ymax, xmax = box
            rect = patches.Rectangle((xmin*300, ymin*300), (xmax-xmin)*300, (ymax-ymin)*300,
                                     linewidth=1, edgecolor='g', facecolor='none')
            plt.gca().add_patch(rect)
        plt.axis('off')
    plt.show()

# Display sample images
show_samples(train_dataset)


# 5. Load Pre-trained Model
We'll use a pre-trained model (SSD MobileNet V2) from TensorFlow Hub.

In [ ]:
# Load a pre-trained object detection model
detector_baseline = hub.KerasLayer("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2", trainable=False)


# 6. Evaluation Function
We will define a function to evaluate the model based on IoU (Intersection over Union).

In [ ]:
# Evaluation function to compute IOU (Intersection over Union)
def calculate_iou(box1, box2):
    y1 = max(box1[0], box2[0])
    x1 = max(box1[1], box2[1])
    y2 = min(box1[2], box2[2])
    x2 = min(box1[3], box2[3])
    intersect = max(0, y2 - y1) * max(0, x2 - x1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersect
    return intersect / union if union > 0 else 0

# Model evaluation function
def evaluate_model(model, dataset):
    total_boxes, correct_boxes = 0, 0
    for image_batch, box_batch, _ in dataset.take(10):
        for image, boxes in zip(image_batch, box_batch):
            image = tf.cast(tf.expand_dims(image * 255, axis=0), dtype=tf.uint8)
            predictions = model(image)
            pred_boxes = predictions['detection_boxes'].numpy()
            true_boxes = boxes.numpy()
            for true_box in true_boxes:
                if not np.any(true_box): continue  # Skip padding
                total_boxes += 1
                for pred_box in pred_boxes[0]:
                    iou = calculate_iou(true_box, pred_box)
                    if iou > 0.5:
                        correct_boxes += 1
                        break
    accuracy = correct_boxes / total_boxes if total_boxes > 0 else 0
    return accuracy

# Evaluate baseline model
baseline_acc = evaluate_model(detector_baseline, val_dataset)
print(f"Baseline Accuracy (IoU > 0.5): {baseline_acc:.2f}")


# 7. Fine-Tuning the Model
If you'd like to fine-tune the model, you can do so as follows:




In [ ]:
# Fine-tune SSD MobileNet V2 (Example)
base_model = tf.keras.applications.MobileNetV2(input_shape=(300, 300, 3), include_top=False, weights='imagenet')
base_model.trainable = True

inputs = tf.keras.Input(shape=(300, 300, 3))
x = base_model(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
box_outputs = tf.keras.layers.Dense(4, name="boxes")(x)
class_outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="classes")(x)

model = tf.keras.Model(inputs, [box_outputs, class_outputs])

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')  # Loss to be adjusted for object detection
model.fit(train_dataset, epochs=5, validation_data=val_dataset)
